<a href="https://colab.research.google.com/github/sambhramnaregal/CAREER_PATH_PREDICTION/blob/main/CareerPathPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Install required libraries
!pip install pandas numpy scikit-learn xgboost joblib


In [4]:
from google.colab import files
import pandas as pd

#uploaded = files.upload("/content/student_career_path_synthetic.csv")   # Upload your CSV (student_career_path_synthetic.csv)
df = pd.read_csv("student_career_path_synthetic.csv")
df.head()


,Name,USN,Gender,Age,CGPA,Branch_Department,Number_of_Backlogs,Number_of_Internships,Type_of_Internships,Number_of_Publications,Number_of_Projects,Number_of_Certification_Courses,Technical_Skills_Score,Number_of_Hackathons,Soft_Skills_Score,Co_curricular_Activities,Leadership_Roles,Entrepreneur_Cell_Member,Family_Business_Background,Status_after_Graduation
0,Student_1,1RV21CS001,Male,24,8.84,ISE,2,0,Corporate,2,4,4,5,1,4,No,Yes,No,No,Higher Studies
1,Student_2,1RV21CS002,Female,22,9.72,AIML,4,1,Research,1,1,5,1,3,3,Yes,Yes,No,No,Placement
2,Student_3,1RV21CS003,Male,22,9.25,CSD,0,1,Corporate,0,3,3,2,3,5,Yes,Yes,No,No,Higher Studies
3,Student_4,1RV21CS004,Male,24,6.24,CIVIL,0,2,Research,2,2,1,3,4,3,Yes,No,No,No,Placement
4,Student_5,1RV21CS005,Male,18,7.25,CS IOT,2,0,Research,0,3,5,4,3,5,Yes,Yes,No,No,Higher Studies


In [5]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Drop ID-like columns
df = df.drop(columns=['Name', 'USN'])

# Handle categorical data
label_encoders = {}
for col in df.select_dtypes(include='object').columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Features + Target
X = df.drop('Status_after_Graduation', axis=1)
y = df['Status_after_Graduation']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# Random Forest
rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
rf_acc = accuracy_score(y_test, rf_pred)

# XGBoost
xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)
xgb_acc = accuracy_score(y_test, xgb_pred)

print("Random Forest Accuracy:", round(rf_acc*100,2), "%")
print("XGBoost Accuracy:", round(xgb_acc*100,2), "%")

print("\nRandom Forest Report:\n", classification_report(y_test, rf_pred))
print("\nXGBoost Report:\n", classification_report(y_test, xgb_pred))


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [18:38:59] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Random Forest Accuracy: 44.0 %
XGBoost Accuracy: 41.0 %

Random Forest Report:
               precision    recall  f1-score   support

           0       0.50      0.12      0.20        41
           1       0.44      0.89      0.59        44
           2       0.00      0.00      0.00        15

    accuracy                           0.44       100
   macro avg       0.31      0.34      0.26       100
weighted avg       0.40      0.44      0.34       100


XGBoost Report:
               precision    recall  f1-score   support

           0       0.46      0.27      0.34        41
           1       0.41      0.64      0.50        44
           2       0.29      0.13      0.18        15

    accuracy                           0.41       100
   macro avg       0.38      0.35      0.34       100
weighted avg       0.41      0.41      0.38       100



In [7]:
'''# Calculate the accuracy by comparing true labels (y_train) with predicted labels
rftr = accuracy_score(y_train, rf_train_pred)
xgtr = accuracy_score(y_train, xg_train_pred)

print("Random Forest Training Accuracy:", rftr)
print("XGBoost Training Accuracy:", xgtr)'''

'# Calculate the accuracy by comparing true labels (y_train) with predicted labels\nrftr = accuracy_score(y_train, rf_train_pred)\nxgtr = accuracy_score(y_train, xg_train_pred)\n\nprint("Random Forest Training Accuracy:", rftr)\nprint("XGBoost Training Accuracy:", xgtr)'

In [8]:
import joblib

best_model = rf if rf_acc >= xgb_acc else xgb
model_name = "best_student_path_model.pkl"
joblib.dump(best_model, model_name)
print(f"✅ Saved best model: {model_name}")

# Also save encoders for later use
joblib.dump(label_encoders, "label_encoders.pkl")


✅ Saved best model: best_student_path_model.pkl


['label_encoders.pkl']

In [9]:
import numpy as np

# Example student record (order of features same as df.columns before dropping target)
sample = X_test.iloc[0].values.reshape(1, -1)
predicted = best_model.predict(sample)[0]

# Decode prediction
target_le = label_encoders['Status_after_Graduation']
decoded = target_le.inverse_transform([predicted])[0]

print("Predicted career path:", decoded)


Predicted career path: Placement


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [10]:
from google.colab import files
files.download("best_student_path_model.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
files.download("best_student_path_model.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
# Make sure the model exists in the current working directory
!ls /content/*.pkl


/content/best_student_path_model.pkl  /content/label_encoders.pkl
